```{contents}
```
## Simulation 

In LangGraph, **simulation** refers to the controlled execution of a graph using **synthetic inputs, mocked components, and virtual environments** to evaluate behavior, correctness, safety, and performance **before real deployment**.
It allows engineers to **test, debug, validate, and optimize complex LLM workflows** deterministically.

---

### **1. Why Simulation Is Necessary**

Production LLM systems are:

* Non-deterministic
* Expensive
* Safety-critical
* Hard to debug after deployment

Simulation provides a **laboratory** where failures are cheap and observable.

| Objective    | Benefit                      |
| ------------ | ---------------------------- |
| Correctness  | Verify logic & routing       |
| Safety       | Detect dangerous behavior    |
| Reliability  | Expose edge cases            |
| Cost control | Prevent wasteful calls       |
| Performance  | Measure latency & throughput |

---

### **2. What Is Simulated**

| Component | How It Is Simulated         |
| --------- | --------------------------- |
| LLMs      | Mock LLMs / fixed outputs   |
| Tools     | Stub functions              |
| Users     | Synthetic conversations     |
| Events    | Artificial triggers         |
| Failures  | Forced exceptions, timeouts |
| Scale     | High-volume synthetic runs  |

---

### **3. Core Simulation Mechanisms**

LangGraph enables simulation through:

* **Controlled State Injection**
* **Mock Nodes**
* **Deterministic Execution**
* **Replay & Checkpointing**
* **Trace Collection**

---

### **4. Simulation Workflow**

```
Design Graph
    ↓
Inject Synthetic State
    ↓
Replace Live Nodes with Mocks
    ↓
Run Deterministic Execution
    ↓
Collect Traces & Metrics
    ↓
Validate Invariants & Outcomes
```

---

### **5. Minimal Simulation Example**

```python
from typing import TypedDict
from langgraph.graph import StateGraph, END

class State(TypedDict):
    user_input: str
    result: str

def mock_llm(state):
    return {"result": "approved"}  # deterministic fake output

builder = StateGraph(State)
builder.add_node("llm", mock_llm)
builder.set_entry_point("llm")
builder.add_edge("llm", END)

graph = builder.compile()

# --- Simulation run ---
simulated_input = {"user_input": "request loan"}
output = graph.invoke(simulated_input)
print(output)
```

**Result:**

```
{'user_input': 'request loan', 'result': 'approved'}
```

No LLM cost, no external dependencies, full determinism.

---

### **6. Failure & Chaos Simulation**

Inject faults:

```python
def failing_tool(state):
    raise TimeoutError("Service unavailable")
```

Run simulation to validate:

* retry logic
* fallback paths
* recovery policies

---

### **7. Replay-Based Simulation**

LangGraph checkpoints allow exact replays.

```python
graph.invoke(state, config={"checkpoint": True})
graph.replay(checkpoint_id)
```

Used for:

* regression testing
* debugging production incidents
* verifying fixes

---

### **8. Monte Carlo Simulation for Agents**

```python
for _ in range(1000):
    graph.invoke(randomized_state())
```

Used to estimate:

* convergence probability
* safety violation frequency
* cost distribution

---

### **9. Production Simulation Use Cases**

| Scenario            | Purpose                 |
| ------------------- | ----------------------- |
| New feature rollout | Risk evaluation         |
| Agent policy tuning | Behavioral testing      |
| Security hardening  | Attack surface analysis |
| Cost forecasting    | Budget planning         |
| Scaling tests       | Throughput limits       |

---

### **10. Mental Model**

Simulation turns LangGraph from a **runtime engine** into a **scientific experiment framework**:

> **Hypothesis → Controlled execution → Measurement → Improvement**

This is what enables **safe autonomous systems** at scale.



### Demonstration

In [1]:
# ===========================
# LangGraph Simulation Demo
# ===========================

from typing import TypedDict
from langgraph.graph import StateGraph, END
import random

# ---------- State Definition ----------
class State(TypedDict):
    user_input: str
    decision: str
    retries: int

# ---------- Mock LLM (deterministic) ----------
def mock_llm(state):
    # deterministic "LLM"
    if "loan" in state["user_input"]:
        return {"decision": "approve"}
    return {"decision": "reject"}

# ---------- Failure Injection ----------
def unstable_tool(state):
    # simulate random failure
    if random.random() < 0.3:
        raise TimeoutError("tool failure")
    return {}

# ---------- Recovery Logic ----------
def recovery(state):
    return {"retries": state["retries"] + 1}

# ---------- Router ----------
def router(state):
    if state["retries"] >= 2:
        return END
    return "tool"

# ---------- Graph Construction ----------
builder = StateGraph(State)

builder.add_node("llm", mock_llm)
builder.add_node("tool", unstable_tool)
builder.add_node("recover", recovery)

builder.set_entry_point("llm")
builder.add_edge("llm", "tool")
builder.add_edge("tool", "recover")

builder.add_conditional_edges("recover", router, {
    "tool": "tool",
    END: END
})

graph = builder.compile()

# ---------- Single Simulation Run ----------
print("\n--- Single Simulation ---")
output = graph.invoke({"user_input": "request loan", "decision": "", "retries": 0})
print(output)

# ---------- Monte Carlo Simulation ----------
print("\n--- Monte Carlo Simulation (100 runs) ---")
failures = 0

for _ in range(100):
    try:
        graph.invoke({"user_input": "request loan", "decision": "", "retries": 0})
    except:
        failures += 1

print("Failure rate:", failures, "/ 100")

print("\nSimulation complete.")



--- Single Simulation ---
{'user_input': 'request loan', 'decision': 'approve', 'retries': 2}

--- Monte Carlo Simulation (100 runs) ---
Failure rate: 47 / 100

Simulation complete.
